# Notebook: Calculate Agreement

This notebook is used to calculate the inter-rater agreement using Krippendorf's Alpha.
<br>**Contributors:** [Nils Hellwig](https://github.com/NilsHellwig/) | [Markus Bink](https://github.com/MarkusBink/)

## Packages

In [17]:
from statsmodels.stats import inter_rater as irr
import krippendorff as kd
import pandas as pd
import numpy as np
import glob
import os

## Parameters

In [18]:
ANNOTATED_DATASET_PATH = "../Datasets/annotated_dataset/"
LABEL_CODING = {'NEUTRAL': 3, 'NEGATIVE': 2, 'POSITIVE': 1, 'MIXED': 4}

## Settings

In [19]:
pd.options.display.max_colwidth = 1000

## Code

### 1. Load Annotations

In [20]:
file_list = sorted(glob.glob(ANNOTATED_DATASET_PATH + "*.xlsx"))

In [21]:
# read and concatenate annotator 1's session 1 and 2 data
df_annotator_1 = pd.concat([
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_1.xlsx"),
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_1.xlsx")
])

In [22]:
# rename sentiment column and recode labels
df_annotator_1.rename(columns={'sentiment': 'sentiment_1'}, inplace=True)
df_annotator_1['sentiment_1'] = df_annotator_1['sentiment_1'].map(LABEL_CODING)

In [23]:
# read and concatenate annotator 2's session 1 and 2 data
df_annotator_2 = pd.concat([
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_2.xlsx"),
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_2.xlsx")
])

In [24]:
# rename sentiment column and recode labels
df_annotator_2.rename(columns={'sentiment': 'sentiment_2'}, inplace=True)
df_annotator_2['sentiment_2'] = df_annotator_2['sentiment_2'].map(LABEL_CODING)

In [25]:
# read and concatenate annotator 3's session 1 and 2 data
df_annotator_3 = pd.concat([
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_3.xlsx"),
    pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_3.xlsx")
])

In [26]:
# rename sentiment column and recode labels
df_annotator_3.rename(columns={'sentiment': 'sentiment_3'}, inplace=True)
df_annotator_3['sentiment_3'] = df_annotator_3['sentiment_3'].map(LABEL_CODING)

In [27]:
# concatenate annotator 1,2,3 data
df_all_annotations = pd.concat([
    df_annotator_1[['id','tweet','sentiment_1']], 
    df_annotator_2[['sentiment_2']],
    df_annotator_3[['sentiment_3']]
], axis=1)

In [28]:
# check for missing values in sentiment columns
print(df_all_annotations[df_all_annotations['sentiment_1'].isnull()])
print(df_all_annotations[df_all_annotations['sentiment_2'].isnull()])
print(df_all_annotations[df_all_annotations['sentiment_3'].isnull()])

Empty DataFrame
Columns: [id, tweet, sentiment_1, sentiment_2, sentiment_3]
Index: []
Empty DataFrame
Columns: [id, tweet, sentiment_1, sentiment_2, sentiment_3]
Index: []
Empty DataFrame
Columns: [id, tweet, sentiment_1, sentiment_2, sentiment_3]
Index: []


In [29]:
df_all_annotations = df_all_annotations.reset_index(drop=True)

In [30]:
df_all_annotations

,id,tweet,sentiment_1,sentiment_2,sentiment_3
0,1460589265759482112,"@FrankieLix2020 @andfra9 @GoeringEckardt @ABaerbock @OlafScholz @spdbt @GrueneBundestag @fdpbt @LieblingXhain @ninastahr Ist doch egal, was das RKI sagt, weil.... Ja, FDP, SPD &amp; Grüne, warum eigentlich? Wir Eltern warten immer noch auf eine Erklärung, warum ihr unsere Kinder durchseuchen wollt. Besonders schön wäre eine stichhaltige &amp; plausible Erklärung.",2,4,2
1,1470141403045019904,Alter! Der @Karl_Lauterbach ist ja schon wieder in einer Talkshow. Ich dachte es endet jetzt und er hat zutun! #karllauterbach #annewill,3,2,2
2,1405950770466497024,@Karl_Lauterbach Sie sehen auch so aus.,3,3,3
3,1350126525220314880,"@PaulZiemiak @CDU 2/x »Wenn die Bürger die Wahl hätten zwischen der parlamentarischen Demokratie, bei der die gewählten Abgeordneten Entscheidungen treffen und Gesetze beschließen und direkter Demokratie, bei der die Bürger in Sachfragen durch Volksabstimmungen entscheiden könnten, würden sich nur",3,3,3
4,1443988844500733952,@ArminLaschet Laschet ist als CDU-Parteivorsitzender peinlich. Ein neuer Parteivorsitzender der CDU muss gewählt werden. Laschet soll sofort zurücktreten! @CDU @CDUNRW_de @csu_lt @ArminLaschet @CSU @csu_bt @CDUNRW_de @CDU_CSU_EP @Wirtschaftsrat @CDU_BW @cdu_hessen @cdurlp @welt @FAZ_Politik,2,2,2
...,...,...,...,...,...
1995,1400720644279504896,"@spdde @OlafScholz mit einem Wumms aus der Krise. Sorry, das ist Volksverblödung",2,2,2
1996,1415980314871087104,@manager_magazin Wieviel kriegt @OlafScholz weil er die Konkurrenz und andere Firmen mit Steuer belegt?,3,3,2
1997,1419538832488374016,"@jensteutrine @fdp @c_lindner Immer und immer wird behauptet, Autos seien umweltschädlich und würden den Klimawandel vorantreiben. In Wahrheit ermöglichen Sie Mobilität unabhängig vom sozialen Hintergrund. Hierzu 5 E-Autos:",2,3,4
1998,1436026687548953088,"@Mirko17817058 @sebastiankurz @ArminLaschet Naja... Wir haben CumEx, Rechtswidrige Polizeieinsätze und Fehlerchen in einem Buch... wollen Sie das jetzt wirklich alles gleichwertig betrachten?",3,3,2


### 2. Get Examples for Paper

In [31]:
df_agreement = df_all_annotations.loc[(df_all_annotations['sentiment_1'] == df_all_annotations['sentiment_2']) & (df_all_annotations['sentiment_1'] == df_all_annotations['sentiment_3'])]

Show positive agreement

In [32]:
df_agreement[df_agreement["sentiment_1"] == 1][:10]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
20,1447576229176122880,"#Legalisierung von Hanf bedeutet eine neue Ehrlichkeit in der Drogenpolitik! @OlafScholz , @Larsklingbeil , @EskenSaskia , @KuehniKev , @c_lindner , @ABaerbock , @Wissing",1,1,1
28,1449152380054934016,@23rasm @VQuaschning @spdbt @Die_Gruenen @fdp Auf den Punkt gebracht.,1,1,1
57,1373752135331033088,@Karl_Lauterbach Meine Rückendeckung haben Sie! Machen Sie bitte weiter.,1,1,1
151,1370678455973646080,@AnkeHoffnung @Karl_Lauterbach @grimmsi1 @TOODALOO5 @Richter_Mueller @HeinoStoever @WurthGeorg @philineedbauer @NiemaMovassat @KirstenKappert @MAStrackZi @haucap @CDR_FFM @hanfverband Danke dir für die Liste ! 😉,1,1,1
208,1406177778177921024,@johannesvogel @Otto_Fricke @fdp_nrw War auch offenkundig beim Friseur - optisch online optimiert,1,1,1
219,1407249931660893952,@mathiasschmitz6 @JRehborn @CDU Danke.,1,1,1
296,1433381874504815104,@SWagenknecht Da haben sie vollkommen Recht.,1,1,1
302,1392623124009689088,@Karl_Lauterbach Die Beschreibung der sozialen Ungleichheit neben dem Klimawandel als größte Probleme der nächsten Jahre fand ich stark.,1,1,1
383,1377365001678192896,(Offenbar) Unpopular opinion: Ich mag @ArminLaschet als Persönlichkeit und kann ihn mir als Kanzler durchaus vorstellen.,1,1,1
385,1352606253885891072,@sbierna @SWagenknecht Gestalten von was? Sahra schafft das schon. Der Wille zählt.,1,1,1


Show negative agreement

In [33]:
df_agreement[df_agreement["sentiment_1"] == 2][:10]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
4,1443988844500733952,@ArminLaschet Laschet ist als CDU-Parteivorsitzender peinlich. Ein neuer Parteivorsitzender der CDU muss gewählt werden. Laschet soll sofort zurücktreten! @CDU @CDUNRW_de @csu_lt @ArminLaschet @CSU @csu_bt @CDUNRW_de @CDU_CSU_EP @Wirtschaftsrat @CDU_BW @cdu_hessen @cdurlp @welt @FAZ_Politik,2,2,2
5,1396429761682021888,.@ABaerbock @Die_Gruenen @cducsubt @spdbt @dielinke @fdpbt : So viel Wald vernichtet unser Konsum https://t.co/NzwU03kjXa #vegan #palmoilfree #vegan #cancelanimalag #Erdueberlastungstag,2,2,2
6,1382709511153189120,@Karl_Lauterbach auch die Lehrkraefte werden nicht geimpft. Geht gar nicht! #bildungabersicher,2,2,2
7,1347699309492494080,@afd_wallduern Mit keinem Wort fordert er dieses Sperren aber wieder typisch @AfD lügen verbreiten.,2,2,2
10,1390305016808828928,"@PDominke @AfDimBundestag Pauschale Verunglimpfung ist ein beliebtes Hobby, also kein Grund zur Sorge. Es gab zu allen Zeiten Gruppen, auf die man eingehackt hat im Gefühl der trauten Gemeinsamkeit der rechthaberischen Mehrheit. Ihnen fiel ja auch nichts Anderes zum Thema ein.",2,2,2
12,1435580015090061056,@StBrandner Die besten dummköpfe,2,2,2
13,1425039526565658880,@c_lindner Und was erreicht man mit einem Embargo? Das der kleine Mann in Belarus seinen Job verliert und man hofft dadurch das er dann auf die Straße geht? Also heißt es wir entziehen den Menschen die Lebensgrundlage und hoffe auf dem Umsturz? Embargos bringen nichts außer leid.,2,2,2
17,1436050553218416896,"@sarfeld @ArminLaschet Solange es keine Distanzierung von #Maassen gibt, distanziert sich #Laschet|s auch nicht von den Inhalten der #AfD... Die Distanzierung ist reines Konkurenzdenken... #NazisRaus #LaschetRuecktritt #CDUrausausderRegierung",2,2,2
21,1458744294542618880,"@derspiegel Scholz versagt irgendwie, @Karl_Lauterbach, tun Sie etwas.",2,2,2
25,1408741290838310912,@Zora_211212 @Patrick03046207 @Alice_Weidel Und auch ich wär voller Hass / gäb es Musik nur ohne Bass,2,2,2


Show neutral agreement

In [34]:
df_agreement[df_agreement["sentiment_1"] == 3][:10]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
2,1405950770466497024,@Karl_Lauterbach Sie sehen auch so aus.,3,3,3
3,1350126525220314880,"@PaulZiemiak @CDU 2/x »Wenn die Bürger die Wahl hätten zwischen der parlamentarischen Demokratie, bei der die gewählten Abgeordneten Entscheidungen treffen und Gesetze beschließen und direkter Demokratie, bei der die Bürger in Sachfragen durch Volksabstimmungen entscheiden könnten, würden sich nur",3,3,3
9,1371730424196701952,@Schmidtlepp @jensspahn SpahnScheuer,3,3,3
14,1447496797295975936,"@nureinleser10 @M_van_Laack @Matthias_Kamann @Joerg_Meuthen Will die AfD irgendwann mal regieren oder wozu ist sie da? Sie oder andere sagen doch, dass es eine schwarz-gelb-blaue Mehrheit gibt, wieso werden dann keine Angebote gemacht? Wieso muss sich die CDU ausgerechnet an der AfD orientieren? Gibt doch auch FDP, Freie Wähler, CSU",3,3,3
15,1345337052557156096,@rRockxter @europeika @CDU Was hat denn die CDU mit dem Christentum zu tun? 🤔 https://t.co/ioFmt1Zny6,3,3,3
18,1454724701847227904,@CMDRArchadder @chindogu_io @MathiasPriebe @Die_Gruenen @ABaerbock @fdp @c_lindner @spdde @OlafScholz Bedeutet?,3,3,3
19,1412613683004461056,@Karl_Lauterbach Welche Mutation war das?,3,3,3
23,1442852713792950016,@conny_asf @realjonaswd @frautaufenbach @darlyn1108 @CDU Welche drei Gesetze denn zum Beispiel?,3,3,3
24,1362792259469664000,@BerndPfeiffer3 @CDU @CDA_Bund @RadtkeMdEP Die #CDA war und ist eine Partei in der Partei und hat so maßgeblich zur Sozialdemokratisierung der #CDU beigetragen.,3,3,3
27,1377679697140669952,"@kruse_michael @RoninRebell @SusanneHennig @dieLinke Hebe, wenn Sie erlauben, auf ihren Account ab. Jäger sollte wissen, es gibt Fehlschüsse, u. der Ruf nach einem sozialistischen D gehört dazu. Hatten wir mit DDR, aber da machte auch nur SED Millionen. Richtig ansprechen, nicht gleich schießen. Mir würde sozial-gerechtes D reichen",3,3,3


Show mixed agreement

In [47]:
df_agreement[df_agreement["sentiment_1"] == 4][:30]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
58,1436397401720372992,@Jim_na_Jim @toniturtle1963 @Karl_Lauterbach Die Maske trägt sich sehr gut. Aber die getackerten Bänder sind nicht optimal. Wenn man sie nicht vorsichtig auf- oder absetzt dann lösen sie sich.,4,4,4
62,1425341079730233088,"@paramotor_lutz @Luisamneubauer @ArminLaschet @OlafScholz Wäre ja auch dumm gewesen bei Atom zu bleiben. Das hätte uns nämlich rein vom Gefühl her noch mehr Zeit verschafft, nicht auf die erneuerbaren Energien umzusteigen. Verzögerung ist übrigens die beste Form der Sabotage. Also alles gut so.",4,4,4
90,1432302887892398080,"@HeidiSigrid @PN46PN46 @Markus_Soeder Ich weiß nicht woran es liebt, aber Habeck und Baerbock kommen einfach menschlicher rüber. Der Rest wirkt wie seelenlose Politiker.",4,4,4
140,1431878456376233984,"@_FriedrichMerz Danke, dass Sie jetzt schon zugeben, die Wahl zu verlieren. Ihr Einsehen kommt sehr früh. Hätte ich gerade von Ihnen nicht erwartet.",4,4,4
297,1395392576077843968,"@KX949 @StBrandner ...also bei allen dämlichen Antworten, das hier ist zwar ""gemein"" aber da steckt viel Photoshop?-Verstand dahinter und man muss lachen!",4,4,4
607,1424056040555941888,"@Ekki_OA @SteffiLemke @sasa_s @JuliaKloeckner @SvenjaSchulze68 DANKE für die Information. Falls Du kein Beleidigungs-Spezialist bist -- sehr kreativ mit den ""Bots"" übrigens - zitiere doch den Gegenstand Deines Unwillens !",4,4,4
756,1429041725612429056,"#Afghanistan. Ein Thema, an dem man politisch verzweifeln könnte. @larsklingbeil und @KuehniKev haben dazu einen hörenswerten #podcast gemacht. #diekfrage https://t.co/NLLerQ4pPL",4,4,4
823,1409131113658913024,"@sloetree @pYranhia @Die_Gruenen @cem_oezdemir Dabei sehe ich eher das Problem, dass die gerade laufende Entwicklung in NRW medial nicht genug gewürdigt wird. Zudem bedient die Union unter Laschet ja sehr gut die Ideale der konservativen Wählerschaft.",4,4,4
838,1365905943037432064,@SusanneHennig @Janine_Wissler @dieLinke Glück im Unglück dass eine solche Unsympathin an die Spitze gewählt wurde. Das könnte die Rückenschießerpartei unter die 5% drücken.,4,4,4
861,1470136150459310080,"@AnneWillTalk @annewill @Markus_Soeder @Karl_Lauterbach @KonstantinKuhle @CerstinGammelin @DasErste Das ist ""Klasse"" ,laut Karl Lauterbach reicht 90% Impfquote gegen Omikron nicht...",4,4,4


Show no agreement

In [45]:
df_all_annotations[(df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_2"]) &
             (df_all_annotations["sentiment_2"] != df_all_annotations["sentiment_3"]) &
             (df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_3"])][:20]

,id,tweet,sentiment_1,sentiment_2,sentiment_3
46,1465149850450809088,"@wenig_worte @Markus_Soeder @MPKretschmer Und wer wendet das aktuell mögliche nicht an? Im Übrigen habe ich ein Parlament gewählt und keine MPK, die mir den Ausgang beschränken kann. Die MPK ist kein Organ und das ist auch gut so.",3,1,4
71,1355510193472365056,"@cem_oezdemir @CSU @fdp Was ist das Problem mit Technologieoffenheit? Soll doch das E-Auto gewinnen. Solange es nicht nur gewinnt, weil wir es hochsubventionieren.",3,4,1
96,1437418166938968064,"@phoenix_de @jankortemdb @Linksfraktion @dieLinke Das sehe ich auch so. Nur auf der Oppositionsbank kann die CDU, CSU gesunden. Marionetten der Wirtschaft. 🤑",2,3,4
108,1433202742609293056,@polenz_r @Karl_Lauterbach Selbst Frankreich bekommt das hin,2,3,4
337,1373696065568965120,@CDU @RegSprecher @jensspahn @peteraltmaier @AndiScheuer der hashtag #Notbremse bekommt seit wenigen Stunden eine richtig gute bedeutung #Rücktritt,1,4,3
347,1442444756504743936,@janboehm @CDU Bitte nicht @n_roettgen der wäre tatsächlich wählbar. Könnt ihr nicht machen!!!,3,4,2
525,1425898501368647936,"@PaulZiemiak @MiKellner Das gejammer zeigt deutlich, das der Nagel auf den Kopf getroffen wurde. Ich finde die Aktion großartig!",1,2,4
576,1362133782527115008,"Hoffen wir mal, dass es zwischen denen keinen Deal gibt: @jensspahn Kanzler, @ArminLaschet nur Vorsitzender …",1,3,2
599,1432957234603086080,@paseidel @_Heimatliebe_ @AfDWahlkampf @AfDJetzt @AfD_Muenster @Holger_Lucius @Chickenwing64 @AfD Bleib Mal schön in deiner 🌈Welt🤣🤣🤣🤣,1,3,2
670,1439305960883905024,@faktinator @EskenSaskia @Dzienus @Die_Gruenen @spdbt @Linksfraktion Die Fusstruppen von RRG in Vorfreude der Wahl.,2,4,1


### 3. Sentiment Class Distribution

n positive majority

In [37]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 1) & (df_all_annotations["sentiment_2"] == 1)) |
                    ((df_all_annotations["sentiment_2"] == 1) & (df_all_annotations["sentiment_3"] == 1)) |
                    ((df_all_annotations["sentiment_1"] == 1) & (df_all_annotations["sentiment_3"] == 1)))])

120

n negative majority

In [38]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 2) & (df_all_annotations["sentiment_2"] == 2)) |
                       ((df_all_annotations["sentiment_2"] == 2) & (df_all_annotations["sentiment_3"] == 2)) |
                       ((df_all_annotations["sentiment_1"] == 2) & (df_all_annotations["sentiment_3"] == 2)))])

976

n neutral majority

In [39]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 3) & (df_all_annotations["sentiment_2"] == 3)) |
                       ((df_all_annotations["sentiment_2"] == 3) & (df_all_annotations["sentiment_3"] == 3)) |
                       ((df_all_annotations["sentiment_1"] == 3) & (df_all_annotations["sentiment_3"] == 3)))])

777

n mixed majority

In [40]:
len(df_all_annotations[(((df_all_annotations["sentiment_1"] == 4) & (df_all_annotations["sentiment_2"] == 4)) |
                       ((df_all_annotations["sentiment_2"] == 4) & (df_all_annotations["sentiment_3"] == 4)) |
                       ((df_all_annotations["sentiment_1"] == 4) & (df_all_annotations["sentiment_3"] == 4)))])

87

n no majority

In [41]:
len(df_all_annotations[(df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_2"]) &
             (df_all_annotations["sentiment_2"] != df_all_annotations["sentiment_3"]) &
             (df_all_annotations["sentiment_1"] != df_all_annotations["sentiment_3"])])

40

### 4. Calculate Krippendorff's Alpha

In [42]:
# Rows are the coders (annotators) # of coders
# Columns are the individual items (sentiment of tweet) # of tweets
value_counts = df_all_annotations[["sentiment_1", "sentiment_2", "sentiment_3"]]
value_counts = value_counts.to_numpy().transpose()
kd.alpha(reliability_data=value_counts, level_of_measurement="nominal")

0.614153070237774

### 5. Calculate Fleiss' Kappa

In [43]:
agg = irr.aggregate_raters(df_all_annotations[["sentiment_1", "sentiment_2", "sentiment_3"]])

In [44]:
irr.fleiss_kappa(agg[0], method='fleiss')

0.6140887516963902